# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Jared Vitug
- Miguel Morales
- Phat Ly
- Kevin Mach

# Group Members IDs

- A92083122
- A########
- A########
- A12647584

# Research Question

Can we predict a current winning nba bracket from regular season matchup statistics collected over the last 10 years?
In a best-of-seven NBA playoff series, two teams will matchup against each other consecutively at least four times. This is unlike the regular season schedule in any way, where teams in the same division play at most four times but over the span of an 82 game season. However, we believe that from regular season statistics gathered from the eight current playoff matchups, over the span of the last ten years, we can predict an accurate playoff bracket for the current season’s playoffs. We will look at data such as each team’s winning percentage, number of assists, total offensive rebounds, number of roster changes, etc. to create a model that will tell us which team will win in a playoff series. We will take that model and use it to pick the winners of our bracket.


## Background and Prior Work

With NBA playoffs in full effect, we knew we wanted to research something relative to basketball that was also fun.  We all know sports are near impossible to predict and basketball is no exception - or is it? We began to wonder what if, given a playoff bracket, we could predict each round of the playoffs and thereby figure out who would win the championship.

Each of us watches basketball and knows a few plays can define a game.  Injuries happen, human error occurs, and upsets take place.  However, in the world of sports betting, none of this matters - only numbers matter.  That’s why we decided to find the best combination of regular season matchup stats (between two teams pitted against each other in the playoffs) that can successfully predict a series outcome.

When it comes to the individual statistics, we know that the team which wins the turnover battle usually comes out on top, as does the team with more rebounds and shooting percentage.  However, with all the different play-styles in today’s NBA (iso-ball, backcourt dominance, frontcourt dominance) we want to find out if there are other team stats that can help us predict winners, such as three pointers taken, team fouls, timeouts used, etc.


https://towardsdatascience.com/predicting-nba-winning-percentage-in-upcoming-season-using-linear-regression-f8687d9c0418
https://github.com/COGS108/FinalProjects-Wi18/blob/master/001-FinalProject.ipynb
The above link is to a similar project where someone took 40 years of basketball data from basketball-reference.com and trained the data to predict NBA winning percentage.  The article says that “average age of the players, margin of victory, number of points scored, number of returning players, and number of blocks” were some of the statistics included, which gives us some idea as to which statistics to pull.   The study’s analysis discusses how wrong the model was in over-predicting winning seasons and underpredicting losing seasons with the reason being injuries, trades, and retirements - all things which are hard to predict and difficult to assess from a mathematical perspective.

# Hypothesis


We hypothesize that it is possible to predict the winner of a 7-game playoff series between two NBA teams by analyzing their regular season matchup statistics.
By narrowing our data to historical team matchups, we can examine specific factors that may be pertinent in explaining why one team may have the competitive edge. There are many accessible statistics for every NBA game, and this data can be utilized to determine if one franchise may have a playstyle better suited for a particular opponent. Features such as the record of the matchups (giving more weight to recent wins), number of possessions, offensive rebounds, total assists, turnovers, etc. are all telling of how advantageous a team is over the other. Although teams change over time due to player, coach, and management contracts, we will attempt to take those conditions into account by feature engineering, as winning teams tend to have less structure change. Additionally, we can favor more recent data as it will be more telling of the current matchup. Therefore, with a catalog of in depth matchup statistics, we hypothesize that we can create a strong dataset for our model to predict a correct playoff bracket.


# Dataset(s)

Dataset Name: NBA Advanced Stat
Dataset Source : https://stats.nba.com/
Dataset Link  : https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season=2018-19&SeasonType=Regular%20Season  	
Number of datasets planned : 10 total, 1 for each regular season starting from August 2008 -  April 2019
 
Dataset Explanation: This website contains datasets about the NBA teams’ statistics from year 1999 to present date. Those datasets are organized as in form of a table, where the rows represent the teams and the columns are observations such as win-loss record, shooting percentages, rebounds ..etc.  Since we are planning on using the statistic of the past 10 regular season matchups, this website is useful in that it contains the information from the previous years and we will have to select the options provided in the website (in the provided link) to get the optimal dataset for our project. We will try to pull the regular season information for each year (from 2008 to present date), then drop off teams that are not participating in the current year playoff. We might also need to differentiate each season statistics, perhaps giving more weight to recent year stats than older years.  


# Setup

In [17]:
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import bs4
from bs4 import BeautifulSoup



In [18]:
#example URL
# https://www.basketball-reference.com/teams/MIL/2019/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8

year_array = ["2010", "2011", "2012", "2013", "2014", "2015", 
              "2016", "2017", "2018", "2019"]
team_array = ["GSW", "HOU", "POR", "DEN", "LAC", "UTA", "OKC", 
             "SAS", "MIL", "BOS", "PHI", "TOR", "DET", "IND", "ORL", "BRK"]

def create_url(year_array, team_array):
    #variables for appending the URLs
    begLink = "https://www.basketball-reference.com/teams/"
    endLink = "/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8"
    #array to store the url's
    #this array has '1' as parameter because numpy arrays
    #need some value to store - keep this in mind that the first
    #index with an actual url is index 1 (through 160)
    url_list = np.array(1)
    #loop through team_array 
    for i in team_array:
        #loop through each year in 
        for j in year_array:
            team_name = i
            # if the team is Brooklyn and the year is 2010-2012, team name
            # is New Jersey Nets
            if(i == "BRK" and (j == "2010" or j == "2011" or j == "2012")):
                team_name = "NJN"    
            temp_url = begLink + team_name + "/" + j + endLink
            #print(temp_url)
            url_list = np.append(url_list, temp_url)
    #print(url_list)
    return url_list

            
url_list = create_url(year_array, team_array)

['1'
 'https://www.basketball-reference.com/teams/GSW/2010/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2011/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2012/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2013/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2014/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2015/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2016/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8'
 'https://www.basketball-reference.com/teams/GSW/2017/gamelog/?f

In [20]:
#array to store individual dataframes before combining
df_array = np.array(1)
for url in url_list:
#    df = pd.read_html(url)
    if url != "1":
        print(url)
        #df = pd.read_html(url)
        

https://www.basketball-reference.com/teams/GSW/2010/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2011/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2012/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2013/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2014/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2015/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2016/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8
https://www.basketball-reference.com/teams/GSW/2017/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkP

In [ ]:
#This is just a sample url. 
game = "https://www.basketball-reference.com/teams/GSW/2010/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8"
page = requests.get(game)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#Get the headers of the table
#The table has two rows, the first row which does not impact the column, the second row does
table = soup.findAll('tr', limit = 2 )
firstRow  = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
secondRow = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*